## Scenario: Unregistered issuers cannot publish lists

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import random

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /health)

In [3]:
health_endpoint = f"{BASE_URL}/health"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")
tc.assertEqual("OK", response.json().get("status"))

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/health
Health check successful! Server is running.


### Issuer A registers on the Status List Server but Issuer B does not

In [4]:
credentials_endpoint = f"{BASE_URL}/credentials"
print(f"Testing POST: {credentials_endpoint}")

gondwana_digital_pole = fixtures.get_gondwana_digital_pole_issuer()
print("Proceeding with Issuer:", gondwana_digital_pole.get('label'))

payload = {
  "issuer": gondwana_digital_pole.get('label'),
  "public_key": gondwana_digital_pole.get('keypair').get('public_key'),
  "alg": gondwana_digital_pole.get('keypair').get('alg')
}

response = requests.post(credentials_endpoint, json=payload)
tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
print("Credentials published successfully!")

Testing POST: http://localhost:8000/credentials
Proceeding with Issuer: gondwana-digital-pole-jgb33
Credentials published successfully!


### Issuer A publishes a status list

In [5]:
status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

bearer_token = fixtures.create_bearer_jwt_token(gondwana_digital_pole)
status_list_id = str(uuid.uuid4())

response = requests.post(status_publish_endpoint, json={
    "list_id": status_list_id,
    "status": [{"index": 1, "status": "VALID"}]
}, headers={"Authorization": f"Bearer {bearer_token}"})

tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")
print(f"Issuer A published status list {status_list_id}")

Testing POST: http://localhost:8000/statuslists/publish
Issuer A published status list 98e456d8-b03b-4c3a-aae7-c199b13f5201


### Issuer B fails to publish a status list because UNREGISTERED

In [6]:
print(f"Testing POST: {status_publish_endpoint}")

scott_holdings = fixtures.get_scott_holdings_issuer()
bearer_token = fixtures.create_bearer_jwt_token(scott_holdings)
status_list_id = str(uuid.uuid4())

response = requests.post(status_publish_endpoint, json={
    "list_id": status_list_id,
    "status": [{"index": 1, "status": "VALID"}]
}, headers={"Authorization": f"Bearer {bearer_token}"})

tc.assertEqual(response.status_code, 401, "Unregistered clients should be unable to publish lists")
tc.assertEqual("Issuer not found", response.text)
print(f"Unregistered Issuer B was prevented from publishing status list {status_list_id}")

Testing POST: http://localhost:8000/statuslists/publish
Unregistered Issuer B was prevented from publishing status list 7f933386-57e8-41b5-90ba-f0efc3d820f9


### Unauthenticated clients are equally prevented from publishing lists

In [7]:
print(f"Testing POST: {status_publish_endpoint}")

response = requests.post(status_publish_endpoint, json={
    "list_id": str(uuid.uuid4()),
    "status": [{"index": 1, "status": "VALID"}]
})

tc.assertEqual(response.status_code, 401, "Authentication should be required on the publish endpoint")
tc.assertEqual("Missing Authorization header", response.text)
print(f"The unauthenticated client was prevented from publishing a status list")

Testing POST: http://localhost:8000/statuslists/publish
The unauthenticated client was prevented from publishing a status list
